# EVAC-2 Code And Details
This notebook contains all code and explanations of said code. Code is separated into blocks, organised and split by purpose of code. 

The method used for this assessment to evolve agents throughout gameplay was a neural network, where weights of said neural network are evolved generation by generation. Further detail can be found at the relevant code sections within this report.

## Agent Representation
As each agent is required to be part of a group, groups were defined as an integer with representation as follows:
* `0` - Saints
* `1` - Buddies
* `2` - Fight Club
* `3` - Vandals

An agent was decided to be represented using the class `Player`, this class could then be used to keep track of all important agent attributes:
* `wealth` - The total wealth of the agent.
* `startingGroup` - The initial group assignment of the agent, reassigned each generation start. This is randomly allocated when the agent is first created.
* `group` - The current group assignment of the agent.
* `gameCount` - The number of games played by the agent.
* `weights` - List of floats used in the neural network to make a decision on which group to join. Evolved by the evolutionary algorithm.
* `fitness` - Current fitness of the agent. 

The game can be played by calling the `getPayoff()` method with the opponent as the required parameter. This is called for each of the two agents chosen in a single game as each agent assumes the role of opponent to the other. Representing each group as an integer allows for very simple calculation of payoffs, with a lookup table implemented for each agent as follows: `[[4,0,4,0],[6,4,6,1],[4,0,1,0],[6,1,6,0]]`. This lookup table stores the payoffs for each group interaction with each group, for a total of 16 different possible interactions. 

The evolutionary aspect of the player class is called with `evaluate()`, with the required parameters of the current game opponent along with an instance of the neural network class. This method loads the agent weights into the neural network, calculates the group that the agent should be assigned to (this is the agent deciding if it should move group) and then the agent fitness is assigned as the agent wealth divided by the game count. This fitness was chosen to allow for a fair comparison between agents regardless of how many games they were randomly chosen to play.

The `reset()` method is called to setup an agent ready for a new generation. This resets the group to the agents starting group, along with setting the agent wealth, fitness, and game counter to 0.

In [239]:
import numpy as np

class Player():
  '''
  stores all data required by one agent
  '''
  def __init__(self, IND_SIZE):
    '''
    initialises an agent ready to play a game 

    args:
      int IND_SIZE - size of neural network, therefore number of weights required
    '''
    self.wealth = 0
    # starting group randomly selected
    self.startingGroup = random.randint(0,3)
    # current group agent is a member of 
    self.group = self.startingGroup
    self.gameCount = 0
    # weights to be used in the neural network, randomly initially assigned
    self.weights = tools.initRepeat(list, toolbox.attr_float, IND_SIZE)
    self.fitness = 0
  
  def evaluate(self,opponent,network):
    '''
    calculate agent group decision and fitness

    args:
      Player opponent - player instance of the opponent this agent is playing against
      NeuralNetwork - network - neural network instance to apply weights to and make group decision
    '''
    # set agent weights in network
    network.setWeightsLinear(self.weights)
    # get group output from neural network
    output = network.feedForward([self.wealth, self.group, opponent.wealth, opponent.group])
    self.group = np.argmax(output, axis=0)
    # set fitness as agent average payoff
    self.fitness = self.wealth / self.gameCount

  def addPayoff(self, opponent):
    '''
    calculate agent payoff, add to wealth and increment game counter

    args:
      Player opponent - player instance of the opponent this agent is playing against
    '''
    # payoff lookup table
    payoffs = [[4,0,4,0],[6,4,6,1],[4,0,1,0],[6,1,6,0]]
    # add value from relevant value in lookup table
    self.wealth += payoffs[self.group][opponent.group]
    self.gameCount += 1

  def reset(self):
    '''
    reset agent ready for the next generation
    '''
    self.wealth = 0
    self.gameCount = 0
    self.group = self.startingGroup
    self.fitness = 0

## Agent Evolution Environment

### Neural Network
The neural network used by all agents in the system is a single hidden layer network with 4 input nodes, 8 hidden nodes and 4 output nodes. These 4 output nodes represent the decision of which group to join, and a group is selected using a `softmax` function. The input nodes of this neural network are the agents own group, the agents own wealth, the opponents group and the opponents wealth. These inputs were settled on after experimentation with other inputs such as the number of games the agent had played, or the current game number. These additional inputs did not improve performance and were removed to reduce chance of overfitting.

A bias of 1 was added to the input layer to improve performance.

In [240]:
import numpy as np

# layer node count
numInputNodes = 4
numHiddenNodes = 8
numOutputNodes = 4

# number of weights required by neural network
IND_SIZE = ((numInputNodes+1) * numHiddenNodes) +  + (numHiddenNodes * numOutputNodes)

class NeuralNetwork(object):
  '''
  neural network for decision on which group agent should join
  '''
  def __init__(self, numInput, numHidden, numOutput):
    '''
    initialises neural network ready for use

    args:
      int numInput - number of input nodes
      int numHidden - number of hidden nodes in single hidden node layer
      int numOutput - number of output nodes (1 per group)
    '''
    self.numInput = numInput + 1
    self.numHidden = numHidden
    self.numOutput = numOutput

    # generate some basic weights for the network
    self.wh = np.random.randn(self.numHidden, self.numInput) 
    self.wo = np.random.randn(self.numOutput, self.numHidden)

    # rectified linear unit to be used in hidden layer
    self.ReLU = lambda x : max(0,x)

  def softmax(self, x):
    '''
    used to normalise output layer

    args:
      list (float) x - output layer of neural network
    return:
      softmax of x
    '''
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

  def feedForward(self, inputs):
    '''
    run inputs through neural network

    args:
      list inputs - neural network inputs
    return:
      softmax output of network
    '''
    # add data to input layer
    inputsBias = inputs[:]
    inputsBias.insert(len(inputs), 1)

    # feed input layer to hidden layer
    h1 = np.dot(self.wh, inputsBias)
    h1 = [self.ReLU(x) for x in h1]

    # feed hidden layer to output layer
    output = np.dot(self.wo, h1)
    return self.softmax(output)

  def setWeightsLinear(self, Wgenome):
    '''
    set given weights for neural network

    args:
      list (float) Wgenome - list of weights evolved by an agent
    '''
    numWeights_IH = self.numHidden * (self.numInput)
    # set hidden layer weights
    self.wh = np.array(Wgenome[:numWeights_IH])
    self.wh = self.wh.reshape((self.numHidden, self.numInput))
    # set output layer weights
    self.wo = np.array(Wgenome[numWeights_IH:])
    self.wo = self.wo.reshape((self.numOutput, self.numHidden))

### Running the Simulation
The game simulation is defined as: for a set number of games, two agents are selected at random from a population to play against each other. Payoffs are calculated and each agent is then given the opportunity to change groups. 

This is implemented within the method `playGameAndEvolve` using the constant `NGAMES` to define the number of games played, and `POP` to specify the population size. The two agents are selected using the code `random.sample(range(POP),2)`, which selects two distinct population indexes, these agents then calculate their respective payoffs using `addPayoff`, and calculate their fitness and decision on if they should migrate groups using `evaluate`. Details of these methods are found in the Agent Representation part of this code.

`playBasicGame` is an additional method that can be used to run the game without allowing agents to swap their group assignments, this will be used to evaluate the behaviour developed through adaption.

In [241]:
# set game number and population
NGAMES = 20000
POP = 1000

def playGameAndEvolve(pop, network):
  '''
  play NGAMES number of games, allowing agents to make decisions on when to change groups using the neural network

  args:
    list (Player) pop - population of agents
    NeuralNetwork network - neural network to use to evaluate
  return:
    population
  '''
  for r in range(NGAMES):
    # randomly select two distinct agents
    selection = random.sample(range(POP),2)
    # calculate each agent payoff
    pop[selection[0]].addPayoff(pop[selection[1]])
    pop[selection[1]].addPayoff(pop[selection[0]])
    # give each agent the opportunity to change groups
    pop[selection[0]].evaluate(pop[selection[1]],network)
    pop[selection[1]].evaluate(pop[selection[0]],network)
  return pop

def playBasicGame(pop):
  '''
  play NGAMES number of games, do not use the neural network or allow agents to change groups

  args:
    list (Player) pop - population of agents
  return:
    population
  '''
  for r in range(NGAMES):
    # randomly select two distinct agents
    selection = random.sample(range(POP),2)
    # calculate each agent payoff
    pop[selection[0]].addPayoff(pop[selection[1]])
    pop[selection[1]].addPayoff(pop[selection[0]])
  return pop

## Agent Adaptation Procedure (Training through Evolution)
Below are some helpful methods to keep track of population performance through evolution. `countGroups()` gets number of agents in each group, while `fitnessStats()` gets the mean, minimum and maximum fitness values for the whole population.

In [242]:
def countGroups(pop):
  '''
  get the total number of agents in each group

  args:
    list (Player) pop - population of agents
  return:
    int list of counts of each group
  '''
  groupTotals=[0,0,0,0]
  for person in pop:
    groupTotals[person.group] += 1
  return groupTotals

def fitnessStats(pop):
  '''
  calculate fitness statistics of the population

  args:
    list (Player) pop - population of agents
  return:
    dictionary of mean, max, min of agent fitness
  '''
  fitnessSum = 0
  fitnessMax = 0 
  fitnessMin = -1
  for person in pop:
    # set highest fitness
    if(person.fitness > fitnessMax):
      fitnessMax = person.fitness
    # set lowest fitness
    if(person.fitness < fitnessMin or fitnessMin == -1):
      fitnessMin = person.fitness
    # sum fitness to be divided for mean
    fitnessSum += person.fitness
  return { "mean": fitnessSum/len(pop), "max":fitnessMax, "min":fitnessMin }

### Evolutionary training method
To evolve each agents neural network weightings, the python module `DEAP` has been used. `DEAP` allows for easy crossover, mutation and selection of the population:
* Crossover - Defined as `mate` within the `DEAP` toolbox, implemented using the `cxOnePoint` method. Probability of crossover for each individual is defined by constant `CXPB` set to `0.1`.
* Mutation - Defined as `mutate` within the `DEAP` toolbox, implemented using the `mutGaussian` method with the parameters `mu=0.0, sigma=0.5, indpb=INDPB`. Probability of an individual being subject to mutation is defined by constant `MUTPB` set to `0.5`, and probability of each chromosome within a selected individual (neural network weight in this implementation) being mutated is defined by constant `INDPB` set to `0.1`. 
* Selection - Defined as `select` within the `DEAP` toolbox, implemented using the `selTournament` method with a tournament size of 3.
* Generations - Number of generations is defined by constant `NGEN` set to `100`.

Once these toolbox commands are setup, a population of constant size `POP` (set to 1000) is generated using `DEAP` where an individual in that population is an instance of the previously discussed `player` class. An instance of the neural network is also initialized to be used by the agents in the population. The game is then played with this initial population and fitness information is output. Generational evolution then begins.

For each generation, tournament selection is used to select a new population of agents based on the fitness of the previous generation. This new population is then subject to crossover and mutation for each agents neural network weights. During mutation an individual also has a chance to mutate its starting group to another randomly selected group. This has the same probability of being mutated as each weight in an agent does. All agents in the population are then reset to their starting values (wealth, group, fitness and number of games played), the game then is played with this new population and fitness information for this new population is output. The next generation then begins.

Using this method, a population uses multi-agent evolution to evolve as each game pits agents against each other.

In [243]:
from deap import base
from deap import tools
import random

# set evolution probabilities 
INDPB = 0.1
CXPB  = 0.1
MUTPB = 0.5
# set number of generations
NGEN  = 100

# setup DEAP toolbox of methods used to evolve
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -1.0, 1.0)
toolbox.register("individual", Player, IND_SIZE)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.5, indpb=INDPB)
toolbox.register("pop", tools.initRepeat, list, toolbox.individual)

# initialise neural network
network = NeuralNetwork(numInputNodes, numHiddenNodes, numOutputNodes)

# setup population
pop = toolbox.pop(n=POP)
print("-- Generation 1 --")
# initial play game, allowing agents to swap groups using neural network
pop[:] = playGameAndEvolve(pop, network)
# get results of game
popFitness = fitnessStats(pop)
# output generation data
print("Fitness avg    :"+str(popFitness["mean"]))
print("Fitness max    :"+str(popFitness["max"]))
print("Group count  : "+str(countGroups(pop)))

# run for all generations
for g in range(2,NGEN+1):
  print("-- Generation %i --" % g)
  # use tournament selection to select offspring
  offspring = toolbox.select(pop, len(pop))
  # clone selected offspring
  offspring = list(map(toolbox.clone, offspring))

  # perform crossover for all combinations of offspring (mating)
  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    # only mate if under crossover probability threshold
    if random.random() < CXPB:
      toolbox.mate(child1.weights, child2.weights)

  # mutate an agent
  for mutant in offspring:
     # only mutate if under mutation probability threshold
    if random.random() < MUTPB:
      toolbox.mutate(mutant.weights)
      # only change group if under independent probability threshold 
      if (random.random() < INDPB):
        mutant.startingGroup = random.randint(0,3)

  # reset all agents in the population
  for person in offspring:
    person.reset()
  
  # play game, allowing agents to swap groups using neural network
  pop[:] = playGameAndEvolve(offspring, network)
  # get results of game
  popFitness = fitnessStats(pop)
  # output generation data
  print("Fitness avg    :"+str(popFitness["mean"]))
  print("Fitness max    :"+str(popFitness["max"]))
  print("Group count  : "+str(countGroups(pop)))


-- Generation 1 --
Fitness avg    :2.740287813822427
Fitness max    :4.9743589743589745
Group count  : [263, 273, 235, 229]
-- Generation 2 --
Fitness avg    :2.43138342730047
Fitness max    :4.28125
Group count  : [138, 474, 82, 306]
-- Generation 3 --
Fitness avg    :3.040486711288989
Fitness max    :4.514285714285714
Group count  : [60, 739, 55, 146]
-- Generation 4 --
Fitness avg    :3.5040598850284423
Fitness max    :4.48
Group count  : [40, 875, 34, 51]
-- Generation 5 --
Fitness avg    :3.639678103787246
Fitness max    :4.444444444444445
Group count  : [27, 905, 35, 33]
-- Generation 6 --
Fitness avg    :3.710721940116971
Fitness max    :4.454545454545454
Group count  : [24, 927, 22, 27]
-- Generation 7 --
Fitness avg    :3.738112760204881
Fitness max    :4.4
Group count  : [24, 935, 20, 21]
-- Generation 8 --
Fitness avg    :3.756242152185986
Fitness max    :4.326530612244898
Group count  : [37, 932, 12, 19]
-- Generation 9 --
Fitness avg    :3.77725236161427
Fitness max    :4.